# Task 1 (6p)
Your task is to modify the custom implementation of MultiHeadAttention. This custom implementation, currently, enables each token to attent to every other token.


Your job is to change this behavior in a specific way.
Let $S$ be our input sequence of length $2 \cdot k$:
- tokens on positions $i \lt k$ should attend to prefix of $S$ of length $k$ ($S[:k]$) - every token up to position k
- tokens on positions $i \ge k$ should attend to prefix of $S$  of length $i + 1$ ($S[:i + 1]$) - every previous token and itself

(Note: You can assume the sequence length is always an even number).

In [164]:
import torch
import math
import torch.nn.functional as F
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_model, num_heads, d_head):
      super().__init__()
      self.d_model = d_model
      self.num_heads = num_heads
      self.d_head = d_head

      self.W_Q = torch.nn.Linear(d_model, num_heads*d_head, bias=True)
      self.W_K = torch.nn.Linear(d_model, num_heads*d_head, bias=True)
      self.W_V = torch.nn.Linear(d_model, num_heads*d_head, bias=True)
      self.W_O = torch.nn.Linear(num_heads*d_head, d_model, bias=True)

    def forward(self, x):

      seq_len, batch_size, _ = x.shape

      Q = self.W_Q(x).reshape(seq_len, batch_size, self.num_heads, self.d_head)
      K = self.W_K(x).reshape(seq_len, batch_size, self.num_heads, self.d_head)
      V = self.W_V(x).reshape(seq_len, batch_size, self.num_heads, self.d_head)

      scaled_QK = torch.einsum("ibhd,jbhd->bhij", Q, K) / math.sqrt(self.d_head)
      # shape of scaled_QK is (batch_size, num_heads, seq_len, seq_len)
      
      #TODO
      pos_k = seq_len // 2
      for i in range(pos_k):
          scaled_QK[:, :, i, pos_k:] = float('-inf')
      for i in range(pos_k, seq_len):
          scaled_QK[:, :, i, i+1:] = float('-inf')
      #ENDTODO
      
      weights = F.softmax(scaled_QK, -1)
      attention = torch.einsum("bhij,jbhd->ibhd", weights, V)

      result = self.W_O(attention.reshape(seq_len, batch_size,self.num_heads * self.d_head))

      return result, weights

In [165]:
# Test your solution
d_model = 8
num_heads = 2
d_head = 4
k = 4
batch_size = 2

mha = MultiHeadAttention(d_model, num_heads, d_head)
batched_x= torch.randn((2*k, batch_size, d_model))
with torch.no_grad():
  result, weights = mha(batched_x)
print("Result:", result)
print("Weights:", weights)

Result: tensor([[[ 0.6165,  0.2279, -0.2474, -0.3699, -0.3429,  0.3070, -0.2449,
          -0.4021],
         [ 0.3658,  0.2750, -0.2977, -0.0200,  0.1576, -0.0410, -0.2112,
           0.0496]],

        [[ 0.5868,  0.1048, -0.2370, -0.3896, -0.3943,  0.2168, -0.0643,
          -0.2470],
         [ 0.3489,  0.2874, -0.2876, -0.0153,  0.1794, -0.0389, -0.2040,
           0.0562]],

        [[ 0.6211,  0.2011, -0.2491, -0.3413, -0.3608,  0.2930, -0.2263,
          -0.4227],
         [ 0.3772,  0.2681, -0.3090, -0.0303,  0.1583, -0.0537, -0.2098,
           0.0560]],

        [[ 0.6297,  0.0413, -0.2484, -0.3042, -0.4244,  0.2260, -0.0458,
          -0.4361],
         [ 0.3354,  0.2986, -0.2832,  0.0012,  0.1969, -0.0436, -0.1920,
           0.0652]],

        [[ 0.6037,  0.2619, -0.2329, -0.3774, -0.3723,  0.2733, -0.3207,
          -0.3864],
         [ 0.2830,  0.3667, -0.2325, -0.0649,  0.2315,  0.0099, -0.1814,
           0.0594]],

        [[ 0.6096,  0.2347, -0.2510, -0.3323, -0.395